# Beispiel 3.4: Reaktionsfortschritt bei komplexen Reaktionen
Bearbeitet von Franz Braun

Dieses Beispiel befindet sich im Lehrbuch auf den Seiten 20 - 22. Das hier angewendete
Vorgehen entspricht dem im Lehrbuch vorgestellten Lösungsweg.

Zunächst werden die benötigten Pakete importiert.

In [1]:
### Import
import numpy as np
from scipy.optimize import root
from tabulate import tabulate

In diesem Beispiel wird die Gleichgewichtszusammensetzung der Methanolsynthese betrachtet.

\begin{align*}
&\mathrm{1.\,\, Reaktion} \quad &   \mathrm{CO + 2\,H_2}    &       \leftrightarrow     \mathrm{CH_3OH} \\
&\mathrm{2.\,\, Reaktion} \quad &   \mathrm{CO_2 + 3\,H_2}  &       \leftrightarrow     \mathrm{CH_3OH + H_2O} \\
&\mathrm{3.\,\, Reaktion} \quad &   \mathrm{CO + H_2O}      &       \leftrightarrow     \mathrm{CO_2 + H_2} \\
\end{align*}

Zwei dieser Reaktionen (z.B. R1 und R2) sind linear unabhängig (vgl. Text zu Beispiel 3.4 im Lehrbuch). Deshalb müssen nur die Massenwirkungsgesetze dieser Reaktionen berücksichtigt werden. 

\begin{align*}
K_{p,1} &= \frac{p_\mathrm{CH_3OH}}{p_\mathrm{CO} \, p_\mathrm{H_2}^2}\\
K_{p,2} &= \frac{p_\mathrm{CH_3OH} \, p_\mathrm{H_2O}}{p_\mathrm{CO_2} \, p_\mathrm{H_2}^3}
\end{align*}

Anschließend werden die Gleichgewichtskonstanten und die Reaktionsbedingungen ($T,\,p$) parametriert.

In [2]:
T       = 500       # K
p       = 45        # bar
K_p1    = 12.0646   # bar-2
K_p2    = 0.1678    # bar−2

Analog zum Beispiel 3.3 kann die Definition der Reaktionslaufzahl $\xi$ mit der gegebenen Ausgangszusammensetzung der Produkte angewendet werden:

\begin{align*}
n_\mathrm{CO} &= 1 \,\mathrm{mol} - \xi_1,\\
n_\mathrm{CO_2} &= 0.1 \,\mathrm{mol} - \xi_2,\\
n_\mathrm{H_2} &= 2 \,\mathrm{mol} - 2\,\xi_1 - 3\,\xi_2,\\
n_\mathrm{CH_3OH} &=  \xi_1 + \xi_2,\\
n_\mathrm{H_2O} &= \xi_2,\\
\sum n_i &= 3,1 \,\mathrm{mol} - 2\,\xi_1 - 2\,\xi_2.\\
\end{align*}

Mit
\begin{align*}
p_i &= x_i \, p,\\
x_i &= \frac{n_i}{n},
\end{align*}
ergeben sich folgende Bestimmungsgleichungen:

\begin{align*}
f_1(\xi_1,\xi_2) &= K_{p,1} \, p^2 \, n_\mathrm{CO} \, n_\mathrm{H_2}^2 - n_\mathrm{CH_3OH} \, n^2 \qquad \enspace = 0,\\
f_2(\xi_1,\xi_2) &= K_{p,2} \, p^2 \, n_\mathrm{CO_2} \, n_\mathrm{H_2}^3 - n_\mathrm{CH_3OH} \, n_\mathrm{H_2O} \, n^2 = 0.
\end{align*}

Um die Nullstellen zu bestimmten, werden $f_1(\xi_1,\xi_2)$ und $f_2(\xi_1,\xi_2)$ als Funktion definiert. Anschließend wird mit _scipy.optimize.root_ nach den Nullstellen gesucht.

In [3]:
def f_xi(xi):
    '''
    Umgeformtes Massenwirkungsgesetz
     -> Funktion zur Bestimmung der Nullstellen 
    
    Parameter:
    ----------
    xi : float
        Reaktionslaufzahlen in mol
        xi[0] -> xi_1
        xi[1] -> xi_2
    
    n_i : float
        Stoffmengenanteile in mol

    '''

    xi_1        = xi[0]
    xi_2        = xi[1]

    # Stoffmengenanteile
    n_CO        = 1         -   xi_1
    n_CO2       = 0.1       -   xi_2
    n_H2        = 2         -   2 * xi_1    -     3 * xi_2
    n_CH3OH     =               xi_1        +     xi_2
    n_H2O       =                               xi_2

    n           = 3.1       - 2 * xi_1      - 2 * xi_2 

    # Bestimmungsgleichungen
    f_1 = K_p1 * p**2 * n_CO  * n_H2**2 - n_CH3OH *         n**2
    f_2 = K_p2 * p**2 * n_CO2 * n_H2**3 - n_CH3OH * n_H2O * n**2

    return np.hstack((f_1,f_2))

In [4]:
xi_guess = np.array((0, 0.))          # "initial guess" für xi_1 und xi_2
sol = root(f_xi, xi_guess, tol = 1e-5)  # Lösen mit scypi.optimize.root

''' 
Wenn der solver erfolgreich ist, wird xi ausgegeben.
Sollte das Lösen fehlschlagen, wird die "solver-message" und der "solver-success" ausgegeben.
'''
if sol.success:
    xi_1 = sol.x[0]
    xi_2 = sol.x[1]
    print('Das Lösen war erfolgreich.')
    print('Für die Reaktionslaufzahl 1 wurde der Wert ', np.round(xi_1,4),' mol ermittelt.')
    print('Für die Reaktionslaufzahl 2 wurde der Wert ', np.round(xi_2,4),' mol ermittelt.')
else:
    print(sol.success)
    print(sol.message)

Das Lösen war erfolgreich.
Für die Reaktionslaufzahl 1 wurde der Wert  0.974  mol ermittelt.
Für die Reaktionslaufzahl 2 wurde der Wert  0.0023  mol ermittelt.


Alternativ ist auch hier eine Lösung mit dem NEWTONschen Verfahren, wie in Beispiel 3.3 demonstriert, möglich. Diese Vorgehensweise wird hier aber nicht gezeigt.

Abschließend werden die Stoffmengen im Gleichgewicht bestimmt.

In [5]:
n_CO_eq         = 1     -   xi_1
n_CO2_eq        = 0.1   -   xi_2
n_H2_eq         = 2     -   2 * xi_1        -     3 * xi_2
n_CH3OH_eq      =               xi_1        +     xi_2
n_H2O_eq        =                                 xi_2

In [6]:
# Ausgabe der Ergebnisse als Tabelle

names_y     = np.array((['n_i / mol']))
n_row       = np.array((n_CO_eq, n_CO2_eq, n_H2_eq, n_CH3OH_eq, n_H2O_eq))  

# Array für die Tabelle
table       = [np.hstack((names_y, np.round(n_row, 4)))]

print(tabulate(table, headers = ['CO', 'CO2', 'H2', 'CH3OH', 'H2O'] ))

              CO     CO2     H2    CH3OH     H2O
---------  -----  ------  -----  -------  ------
n_i / mol  0.026  0.0977  0.045   0.9763  0.0023
